In [1]:
import kagglehub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import glob as gb
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# **Load dataset**

In [2]:
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")
print("Path to dataset files:", path)

100%|██████████| 2.70G/2.70G [00:30<00:00, 94.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2


# **Train, Validation, and Test directories**

In [3]:
train_data = '/root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_data = '/root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
test_data = valid_data

# **Data augmentation and loading**

In [4]:
train_size = 224
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
    validation_split=0.1
).flow_from_directory(
    train_data,
    target_size=(train_size, train_size),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 63282 images belonging to 38 classes.


In [5]:
valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.1
).flow_from_directory(
    valid_data,
    batch_size=164,
    target_size=(train_size, train_size),
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 1742 images belonging to 38 classes.


In [6]:
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0
).flow_from_directory(
    test_data,
    batch_size=64,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=False
)

Found 17572 images belonging to 38 classes.


# **Transfer Learning Model (EfficientNetB0)**

In [7]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# **Building the new model**

In [8]:
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(38, activation='softmax')
])

# **Learning rate scheduling**

In [9]:
initial_learning_rate = 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100, decay_rate=0.96, staircase=True
)

# **Compile the model**

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# **Callbacks**

In [11]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
model_ReduceLRonPlateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, min_lr=0.00001)
callbacks = [early_stopping, model_checkpoint, model_ReduceLRonPlateau]

# **Train the model**

In [ ]:
history = model.fit(train_generator, epochs=5, validation_data=valid_generator, callbacks=callbacks)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
989/989 ━━━━━━━━━━━━━━━━━━━━ 5926s 6s/step - accuracy: 0.0250 - loss: 3.7726 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.0287 - val_loss: 3.6432 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 6.9253e-04
Epoch 2/5
188/989 ━━━━━━━━━━━━━━━━━━━━ 1:16:46 6s/step - accuracy: 0.0312 - loss: 3.6434 - precision: 0.0000e+00 - recall: 0.0000e+00

# **Evaluate model on test data**

In [ ]:
model_evaluate = model.evaluate(test_generator)
print("Loss:", model_evaluate[0])
print("Accuracy:", model_evaluate[1])
print("Precision:", model_evaluate[2])
print("Recall:", model_evaluate[3])

#**Save the model**

In [ ]:
model.save('CNN_Model.keras')